### import

In [ ]:
import scanpy as sc
import scirpy as ir
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import random
random.seed(42)

In [ ]:
def gini(x):
    total = 0
    sorted_x = np.sort(x)
    for i, xi in enumerate(sorted_x[:-1], 1):
        total += np.sum(np.abs(xi - sorted_x[i:]))
    return total / (len(sorted_x)**2 * np.mean(sorted_x))

def pivot_table(v1, v2, N='N'):
    """
    Aggregates two identically indexed pd.Series into a table with amount of pairs (v1.x, v2.y) in a cell
    :param v1: pd.Series
    :param v2: pd.Series
    :return: pd.DataFrame pivot table
    """

    sub_df = pd.DataFrame({'V1':v1,
                           'V2':v2})
    sub_df['N'] = 1

    return pd.pivot_table(data=sub_df, columns='V1',
                          index='V2', values='N', aggfunc=sum).fillna(0).astype(int)

In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=200, frameon=True, vector_friendly=True, fontsize=14,
                     figsize=(4,4),
                     color_map=None, format='png',
                     facecolor=None, transparent=False, ipython_format='png2x')

cluster_colors = {
    'Th1-cytotoxic': '#762a83',
    'Effector/Th17': '#B7C6E1',
    'Early effector memory': '#C4E8E4',
    'Transitional memory': '#c4b8a7',
    'Central memory': '#FFC285',
    'Tfh/Tfr-like': '#182230',
    'Type I IFN signature': '#d92e61',
    'Th2': '#0000ff'
}

diagnosis_colors = {'CD':'#F80106', 'HD':'#484748'}

## Data preprocessing 

### load GEX and TCR files

In [ ]:
samples_list = [
    'Calbicans_2',
    'Calbicans_20211012CDpatient',
    'Calbicans_20211018Healthy3',
    'Calbicans_20211020CDpatient2',
    'Ctropicalis_2',
    'Ctropicalis_20211012CDpatient',
    'Ctropicalis_20211018Healthy3',
    'Ctropicalis_20211020CDpatient2',
    'Scerevisiae_2',
    'Scerevisiae_20211012CDpatient',
    'Scerevisiae_20211018Healthy3',
    'Scerevisiae_20211020CDpatient2',
    'Calbicans_20211115Healthy4',
    'Scerevisiae_20211115Healthy4',
    'Ctropicalis_20211115Healthy4',
    'Fungi_CD3_Calbicans',
    'Fungi_CD3_Ctropicalis',
    'Fungi_CD3_Scerevisiae'
               ]

fungi_adatas = []
for sample in samples_list:
    print(sample)
    adata = sc.read_10x_h5(f'/mnt/medcluster/10x_raw/fungi_h5ad/{sample}/filtered_feature_bc_matrix.h5',gex_only=False)
    adata.var_names_make_unique()
    #CITE
    protein = adata[:, adata.var["feature_types"] == "Antibody Capture"].copy()
    hash_df = protein.to_df().idxmax(axis=1).map({'HASHTAG1':'IFNG',
    'HASHTAG2':'DN',
    'HASHTAG3':'IL17',
     'IFNG':'IFNG',
    'IFNg':'IFNG',
    'IL17':'IL17',
    'DN':'DN'
            })
    adata.obs['cell_subtype'] = hash_df
    #TCR
    adata_tcr = ir.io.read_10x_vdj(f'/mnt/medcluster/10x_raw/fungi_tcr/{sample}/filtered_contig_annotations.csv')
    ir.pp.merge_with_ir(adata, adata_tcr)
    fungi_adatas.append(adata)
    

fungi = fungi_adatas[0].concatenate(fungi_adatas[1:], batch_categories=samples_list+new_samples_list+newest_samples_list)

In [ ]:
antigen_species_map = {
 'Calbicans_2':'Calbicans',
 'Calbicans_20211012CDpatient':'Calbicans',
 'Calbicans_20211018Healthy3':'Calbicans',
 'Calbicans_20211020CDpatient2':'Calbicans',
 'Ctropicalis_2':'Ctropicalis',
 'Ctropicalis_20211012CDpatient':'Ctropicalis',
 'Ctropicalis_20211018Healthy3':'Ctropicalis',
 'Ctropicalis_20211020CDpatient2':'Ctropicalis',
    'Ctropicalis_20211115Healthy4':'Ctropicalis',
 'Scerevisiae_2':'Scerevisiae',
 'Scerevisiae_20211012CDpatient':'Scerevisiae',
 'Scerevisiae_20211018Healthy3':'Scerevisiae',
 'Scerevisiae_20211020CDpatient2':'Scerevisiae',
 'DHanseni_2':'DHanseni',
 'Dhanseni_20211018Healthy3':'DHanseni',
 'Calbicans_20211115Healthy4':'Calbicans',
 'Fungi_CD3_Calbicans':'Calbicans',
 'Fungi_CD3_Ctropicalis':'Ctropicalis',
 'Fungi_CD3_Scerevisiae':'Scerevisiae',
          'Scerevisiae_20211115Healthy4':'Scerevisiae',
    'Dhanseni_20211115Healthy4':'DHanseni'
}
fungi.obs['antigen_species'] = fungi.obs['batch'].map(antigen_species_map)

In [ ]:
donor_map = {
 'Calbicans_2':'HD1',
 'Calbicans_20211012CDpatient':'CD1',
 'Calbicans_20211018Healthy3':'HD2',
 'Calbicans_20211020CDpatient2':'CD2',
 'Ctropicalis_2':'HD1',
 'Ctropicalis_20211012CDpatient':'CD1',
 'Ctropicalis_20211018Healthy3':'HD2',
 'Ctropicalis_20211020CDpatient2':'CD2',
 'Scerevisiae_2':'HD1',
 'Scerevisiae_20211012CDpatient':'CD1',
 'Scerevisiae_20211018Healthy3':'HD2',
 'Scerevisiae_20211020CDpatient2':'CD2',
 'DHanseni_2':'HD1',
 'Dhanseni_20211018Healthy3':'HD2',
 'Calbicans_20211115Healthy4':'HD3',
'Ctropicalis_20211115Healthy4':'HD3',
 'Fungi_CD3_Calbicans':'CD3',
 'Fungi_CD3_Ctropicalis':'CD3',
 'Fungi_CD3_Scerevisiae':'CD3',
       'Scerevisiae_20211115Healthy4':'HD3',
    'Dhanseni_20211115Healthy4':'HD3'
}
fungi.obs['donor'] = fungi.obs['batch'].map(donor_map)
fungi.obs['diagnosis'] = fungi.obs['donor'].apply(lambda x: x[:2])

In [ ]:
sc.write('final_fungi.h5ad', fungi)

In [ ]:
fungi = sc.read_h5ad('final_fungi.h5ad')

### QC

In [ ]:
fungi.var['mt'] = fungi.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
fungi.var['ribo'] = fungi.var_names.str.startswith(("RPS","RPL")) # ribosomal genes
fungi.var['hb'] = fungi.var_names.str.contains(("^HB[^(P)]")) # hemoglobin genes

sc.pp.calculate_qc_metrics(fungi, qc_vars=['mt','ribo','hb'], percent_top=None, log1p=False, inplace=True)
# plot QC

sc.pl.violin(fungi, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
initial_cells_n = fungi.shape[0]
fungi = fungi[fungi.obs.pct_counts_mt < 5, :]
fungi = fungi[fungi.obs.n_genes_by_counts > 500, :]
fungi = fungi[fungi.obs.total_counts > 1500, :]

fungi.layers['counts'] = fungi.X
fungi.layers["counts_csc"] = fungi.layers["counts"].tocsc()
cells_left = fungi.shape[0]

print(f'{cells_left}/{initial_cells_n} cells passed the QC filters')

In [ ]:
# Normalize and log transform
sc.pp.normalize_total(fungi, target_sum=1e4)
sc.pp.log1p(fungi)

In [ ]:
# Identify highly variable genes and scale by gene
sc.pp.highly_variable_genes(fungi, min_mean=0.0125, max_mean=10, min_disp=0.3)
sc.pl.highly_variable_genes(fungi)

# remove TCR genes from highly variable
fungi.var['TCR'] = fungi.var_names.str.contains(("TR[ABGD][VJ]"))
fungi.var['highly_variable'] = fungi.var.highly_variable&~fungi.var.TCR

fungi.raw = fungi
fungi = fungi[:, fungi.var.highly_variable]

#sc.pp.regress_out(fungi, ['total_counts', 'pct_counts_mt'], n_jobs=15)
sc.pp.scale(fungi, max_value=10, )

### Batch correction with harmony

In [ ]:
sc.tl.pca(fungi, svd_solver='arpack', use_highly_variable=True)
sc.pp.neighbors(fungi)
sc.tl.umap(fungi)
sc.pl.umap(fungi, color=['batch', 'donor','antigen_species'], ncols=1)

In [ ]:
# batch correction
sc.external.pp.harmony_integrate(fungi, key='donor') 
sc.pp.neighbors(fungi, use_rep='X_pca_harmony')
sc.tl.umap(fungi)
sc.pl.umap(fungi, color=['batch', 'donor','antigen_species'], ncols=1)

## Clustering 

In [ ]:
sc.tl.leiden(fungi, key_added='leiden', resolution=0.8)
sc.pl.umap(fungi, color='leiden', title='Unsupervised clustering')

In [ ]:
markers = [
    'CD14',
    'CD79A',
    'PRF1',
    'GZMB',
    'SLAMF7',
    'PLEK',
    'CCL3',
    'CCL4',
    'CCL5',
    'IFNG',
    'TBX21',
    'CSF2',
    'IL2',
    'IL21',
    'IL22',
    'IL17A',
    'CCR7',
    'SELL',
    'CD27',
    'ICOS',
    'PDCD1',
    'CXCR5',
    'POU2AF1',
    'CTLA4',
    'FOXP3',
    'LAG3',
    'IL10',
    'MX1',
    'MX2',
    'ISG15',
    'ISG20',
    'OAS1',
    'OAS2',
    'IL4',
    'IL5',
    'IL13',
    'GATA3',

]
sc.pl.dotplot(fungi, var_names=markers, groupby='leiden',standard_scale='var', use_raw=True)

In [ ]:
# remove monocytes and B cells and cluster again
fungi = fungi[fungi.obs.leiden!='12']
sc.pp.neighbors(fungi, use_rep='X_pca_harmony')
sc.tl.paga(fungi)
sc.pl.paga(fungi, plot=False)  
sc.tl.umap(fungi, init_pos='paga')
sc.tl.umap(fungi)
sc.tl.leiden(fungi, key_added='leiden', resolution=0.8)
sc.pl.umap(fungi, color='leiden', title='Unsupervised clustering')

In [ ]:
sc.pl.dotplot(fungi, var_names=markers[2:], groupby='leiden',standard_scale='var', use_raw=True)

In [ ]:
cluster_mapping = {
    '0':'Th1-cytotoxic',
    '1':'Early effector memory',
    '2':'Central memory',
    '3':'Effector/Th17',
    '4':'Transitional memory',
    '5':'Transitional memory',
    '6':'Transitional memory',
    '7':'Central memory',
    '8':'Tfh/Tfr-like',
    '9':'Type I IFN signature',
    '10':'Th2'
}
fungi.obs['Cluster'] = fungi.obs['leiden'].map(cluster_mapping)

In [ ]:
sc.pl.umap(fungi[fungi.obs.diagnosis=='HD'], color=['Cluster'],ncols=1, palette=cluster_colors,
           size=8,
           outline_color=('white','white'),
           title='Healthy', frameon=False)

sc.pl.umap(fungi[fungi.obs.diagnosis=='CD'], color=['Cluster'],ncols=1,
           outline_color=('white','white'),
           size=8,
           palette=cluster_colors, title="Crohn's Disease",frameon=False)

## TCR analysis

In [ ]:
ir.tl.chain_qc(fungi)
ir.pp.ir_dist(
    fungi,
    metric='hamming',
    sequence="aa",
    n_jobs=7,
    cutoff=5,
) # computes distances between CDR3 or amino acid sequences

ir.tl.define_clonotypes(fungi, receptor_arms="VDJ", dual_ir="primary_only") 


In [ ]:
fungi = fungi[(fungi.obs['IR_VDJ_1_junction_aa'].notna())] 


In [ ]:
sc.pl.umap(fungi[(fungi.obs.diagnosis=='HD')&(~fungi.obs.duplicated(['clone_id']))],
           color=['Cluster'],
           palette=cluster_colors,
           size=3*fungi[(fungi.obs.diagnosis=='HD')&(~fungi.obs.duplicated(['clone_id']))].obs.clone_id_size,
       #     save='_clone_size_healthy.pdf',
           add_outline=True,
           outline_width=(0.02,0.01),
           outline_color=('white','white'),
           title='Healthy',
           frameon=False)

sc.pl.umap(fungi[(fungi.obs.diagnosis=='CD')&(~fungi.obs.duplicated(['clone_id']))],
           color=['Cluster'],
      #     save='_clone_size_Crohns.pdf',
           add_outline=True,
           outline_width=(0.02,0.01),
           outline_color=('white','white'),
           size=3*fungi[(fungi.obs.diagnosis=='CD')&(~fungi.obs.duplicated(['clone_id']))].obs.clone_id_size,
           palette=cluster_colors,
           title="Crohn's Disease",
           frameon=False)

### Gini index

In [ ]:
fungi.obs['N'] = 1
gb = fungi.obs.groupby(['diagnosis','donor','Cluster','clone_id']).aggregate('count')['N'].reset_index()

Gini_all = []
for donor in gb.donor.unique():
    gb_donor = gb[(gb['donor']==donor)]
    clone_counts = pd.pivot_table(data=gb_donor, values='N', index='clone_id', columns='Cluster', aggfunc='sum')
    #clone_counts = clone_counts.fillna(0)
    gini_donor=clone_counts.apply(lambda col: gini(col[col>0])).to_frame(name='Gini index')
    gini_donor['donor'] = donor
    gini_donor['diagnosis'] = donor[:2]
    Gini_all.append(gini_donor)
    

Gini_all = pd.concat(Gini_all).reset_index()

In [ ]:
fig,ax = plt.subplots(figsize=(5,5))
sns.set_style('ticks')
sns.boxplot(data=Gini_all, x='Cluster', y='Gini index', hue='diagnosis', palette=diagnosis_colors)
sns.swarmplot(data=Gini_all, x='Cluster', y='Gini index', hue='diagnosis', color='black', dodge=True)
ax.set_xlabel('')
ax.tick_params(axis='x', rotation=90)
ax.set_title('Gini index')

plt.tight_layout()
plt.savefig('figures/Gini_clusters.pdf')

### Cross-reactivity

In [ ]:
clones_cross = fungi.obs.groupby('clone_id')['antigen_species'].unique().apply(lambda x: ' '.join(sorted(x)))
fungi.obs['cross'] = fungi.obs.clone_id.map(clones_cross)

order = [
  #  'Calbicans', 'Ctropicalis', 'Scerevisiae',
    'Calbicans Ctropicalis', 'Calbicans Scerevisiae', 'Ctropicalis Scerevisiae',
    'Calbicans Ctropicalis Scerevisiae'
]
order_diag=[]
for x in order:
    order_diag.append(x+'_HD')
    order_diag.append(x+'_CD')

sns.set_style('whitegrid')
fig, ax = plt.subplots(1,1, figsize=(5,4))
pt = pivot_table(fungi.obs.cross.astype('str')+'_'+fungi.obs.diagnosis.astype('str'), fungi.obs['Cluster'])[order_diag].T


pt.plot(kind='bar',
        stacked=True,
        width=0.9,
        linewidth=0.1,              
        color=cluster_colors,
       ax=ax)
ax.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0.1)
#ax.set_title("Cross-reactive cells")
ax.set_xticklabels(order_diag, rotation=90)
ax.set_xlabel('')

### Data for cytoscape plots

In [ ]:
fungi_nosingle = fungi[fungi.obs.clone_id_size>1]

graph_df = fungi_nosingle.obs.groupby(['antigen_species','clone_id']).agg({'N' : sum,
             'donor': lambda x: x.unique()[0],
                'diagnosis': lambda x: x.unique()[0]})
graph_df=graph_df.reset_index()
graph_df = graph_df[graph_df['N']!=0]


clones_cross = graph_df.groupby('clone_id')['antigen_species'].unique().apply(lambda x: ' '.join(sorted(x)))


clone_ann = fungi_nosingle.obs.groupby(['clone_id']).agg({'N' : sum,
             'cross': lambda x: x.unique()[0]})
clone_ann['cross'] = clones_cross
clone_ann['N'] = clone_ann.N.apply(int)
clone_ann.to_csv('nodes.csv')

In [ ]:
for donor in fungi.obs.donor.unique():
    graph_df[graph_df.donor==donor].set_index('clone_id').to_csv(f'edges_{donor}.csv')

## TCR analysis with downsampling

In [ ]:
downsampled_index = []
for batch in fungi.obs.batch.unique():
    if fungi.obs[fungi.obs.batch==batch].shape[0]>500:
        sample = fungi.obs[fungi.obs.batch==batch].sample(n=500)
    else:
        sample = fungi.obs[fungi.obs.batch==batch]
    downsampled_index.extend(list((sample.index)))
fungi_downsampled = fungi[fungi.obs.index.isin(downsampled_index)]

In [ ]:
ir.tl.chain_qc(fungi_downsampled)
ir.pp.ir_dist(
    fungi_downsampled,
    metric='hamming',
    sequence="aa",
    n_jobs=7,
    cutoff=5,
) # computes distances between CDR3 or amino acid sequences

ir.tl.define_clonotypes(fungi_downsampled, receptor_arms="VDJ", dual_ir="primary_only") 


In [ ]:
sc.pl.umap(fungi_downsampled[(fungi_downsampled.obs.diagnosis=='HD')&(~fungi_downsampled.obs.duplicated(['clone_id']))], color=['Cluster'],ncols=1, palette=cluster_colors,
           size=5*fungi_downsampled[(fungi_downsampled.obs.diagnosis=='HD')&(~fungi_downsampled.obs.duplicated(['clone_id']))].obs.clone_id_size,
            save='_downsampled_clone_size_healthy.pdf',
           add_outline=True,
           
           outline_width=(0.02,0.01),
           outline_color=('white','white'),
           title='Healthy', frameon=False)

sc.pl.umap(fungi_downsampled[(fungi_downsampled.obs.diagnosis=='CD')&(~fungi_downsampled.obs.duplicated(['clone_id']))], color=['Cluster'],ncols=1,
           save='_downsampled_clone_size_Crohns.pdf',
           add_outline=True,
           outline_width=(0.02,0.01),
           outline_color=('white','white'),
           size=5*fungi_downsampled[(fungi_downsampled.obs.diagnosis=='CD')&(~fungi_downsampled.obs.duplicated(['clone_id']))].obs.clone_id_size,
           palette=cluster_colors, title="Crohn's Disease",frameon=False)

In [ ]:
clones_cross = fungi_downsampled.obs.groupby('clone_id')['antigen_species'].unique().apply(lambda x: ' '.join(sorted(x)))
fungi_downsampled.obs['cross'] = fungi_downsampled.obs.clone_id.map(clones_cross)

order = [
  #  'Calbicans', 'Ctropicalis', 'Scerevisiae',
    'Calbicans Ctropicalis', 'Calbicans Scerevisiae', 'Ctropicalis Scerevisiae',
    'Calbicans Ctropicalis Scerevisiae'
]
order_diag=[]
for x in order:
    order_diag.append(x+'_HD')
    order_diag.append(x+'_CD')

sns.set_style('whitegrid')
fig, ax = plt.subplots(1,1, figsize=(5,4))
pt = pivot_table(fungi_downsampled.obs.cross.astype('str')+'_'+fungi_downsampled.obs.diagnosis.astype('str'), fungi_downsampled.obs['Cluster'])[order_diag].T


pt.plot(kind='bar',
        stacked=True,
        width=0.9,
        linewidth=0.1,              
        color=cluster_colors,
       ax=ax)
ax.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0.1)
#ax.set_title("Cross-reactive cells")
ax.set_xticklabels(order_diag, rotation=90)
ax.set_xlabel('')

In [ ]:
sns.set_style('ticks')

fig, axs = plt.subplots(2, 2, figsize=(4, 8))
af = axs.flat

gb = fungi_downsampled.obs.groupby(['donor','antigen_species','Cluster']).aggregate('count')['N'].reset_index()
gb['N_fraction'] = gb.groupby(['donor','antigen_species'])['N'].apply(lambda x: x / x.sum())
gb['diagnosis'] = gb['donor'].apply(lambda x: x[:-1])

for group in ['Th1-cytotoxic','Effector/Th17']:
    ax = next(af)
    sns.swarmplot(data=gb[(gb.Cluster==group)&(gb.diagnosis=='HD')],
                  x='antigen_species', 
                  y='N_fraction',
                   #  hue='antigen_species',
                  color = '#484748',
                  dodge=True,
                  #palette={'CD':'#F80106'', 'HD':'#484748'},
                  ax=ax,
                    size=10)
    sns.despine()
    ax.legend([],[], frameon=False)
    #ax.legend(bbox_to_anchor=(1, 1.05))
    ax.set_ylabel( 'fraction in sample')
    ax.set_title(group)
    ax.set_xlabel('')
    ax.set_ylim(0,0.75)
    ax.tick_params('x', rotation=90)
    #ax.set_title('')
    
    ax = next(af)
    sns.swarmplot(data=gb[(gb.Cluster==group)&(gb.diagnosis=='CD')],
                  x='antigen_species', 
                  y='N_fraction',
                    # hue='antigen_species',
                  color = '#F80106',
                  dodge=True,
                  #palette={'CD':'#F80106'', 'HD':'#484748'},
                  ax=ax,
                    size=10)
    sns.despine()
    ax.legend([],[], frameon=False)
    #ax.legend(bbox_to_anchor=(1, 1.05))
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.set_ylim(0,0.75)
    ax.set_title('')
    ax.tick_params('x', rotation=90)
    ax.get_yaxis().set_visible(False)


plt.tight_layout()
plt.subplots_adjust(hspace=1, wspace=0.01)

In [ ]:
fungi_downsampled.obs['N'] = 1
gb = fungi_downsampled.obs.groupby(['diagnosis','donor','Cluster','clone_id']).aggregate('count')['N'].reset_index()

Gini_all = []
for donor in gb.donor.unique():
    gb_donor = gb[(gb['donor']==donor)]
    clone_counts = pd.pivot_table(data=gb_donor, values='N', index='clone_id', columns='Cluster', aggfunc='sum')
    #clone_counts = clone_counts.fillna(0)
    gini_donor=clone_counts.apply(lambda col: gini(col[col>0])).to_frame(name='Gini index')
    gini_donor['donor'] = donor
    gini_donor['diagnosis'] = donor[:2]
    Gini_all.append(gini_donor)
    

Gini_all = pd.concat(Gini_all).reset_index()

fig,ax = plt.subplots(figsize=(5,5))
sns.set_style('ticks')
sns.boxplot(data=Gini_all, x='Cluster', y='Gini index', hue='diagnosis', palette=diagnosis_colors)
sns.swarmplot(data=Gini_all, x='Cluster', y='Gini index', hue='diagnosis', color='black', dodge=True)
ax.set_xlabel('')
ax.tick_params(axis='x', rotation=90)
ax.set_title('Gini index')

plt.tight_layout()
plt.savefig('figures/downsampled_Gini_clusters.pdf')